In [1]:
#%load_ext autoreload
#%autoreload 2
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GroupKFold, RepeatedKFold
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler, MinMaxScaler

from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.linear_model import Lasso
import lightgbm as lgb
import tensorflow as tf
import xgboost as xgb
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import RidgeCV
from sklearn import svm
from catboost import CatBoostRegressor

### Import Feathers and Files

In [3]:
#current #1
import feather
#Best_Feathers
X_train = pd.read_feather(r'C:\Users\user\Desktop\Kaggle\HousePrices\Best_Feathers\X_train_FE_D_13.feather')
test = pd.read_feather(r'C:\Users\user\Desktop\Kaggle\HousePrices\Best_Feathers\test_FE_D_13.feather')

#csv
y_temp = pd.read_csv(r'C:\Users\user\Desktop\Kaggle\HousePrices\Best_Feathers\y_train_FE_D_13.csv', low_memory = False, header = None, names = 'y')
y_train = y_temp['y']

test_ID_temp = pd.read_csv(r'C:\Users\user\Desktop\Kaggle\HousePrices\test_ID.csv', low_memory = False, header = None, names = 'i')
test_ID = test_ID_temp['i']

#import meta feather
#meta level 1
meta_cols_f = pd.read_feather(r'C:\Users\user\Desktop\Kaggle\HousePrices\Feathers\X_meta100.feather')
test_f = pd.read_feather(r'C:\Users\user\Desktop\Kaggle\HousePrices\Feathers\test_meta100.feather')
#meta level 2
meta_cols_f_2 = pd.read_feather(r'C:\Users\user\Desktop\Kaggle\HousePrices\Feathers\X_meta100_2.feather')
test_f_2 = pd.read_feather(r'C:\Users\user\Desktop\Kaggle\HousePrices\Feathers\test_meta100_2.feather')


print(X_train.shape,"  ",test.shape,"  ",y_train.shape, "  ", test_ID.shape)

(1453, 312)    (1459, 312)    (1453,)    (1459,)


In [4]:
from IPython.display import display

def display_all(df):
    with pd.option_context("display.max_rows", 1000): 
        with pd.option_context("display.max_columns", 1000): 
            display(df)
            
display_all(meta_cols_f_2.head(10).transpose())

,0,1,2,3,4,5,6,7,8,9
LotFrontage,18.144572,20.673625,18.668046,17.249650,21.314282,21.472480,19.854050,20.673625,15.558090,15.362900
LotArea,13.833055,14.117918,14.476513,14.106197,15.022009,14.998258,14.228590,14.294345,13.126989,13.546136
OverallQual,7.000000,6.000000,7.000000,7.000000,8.000000,5.000000,8.000000,7.000000,7.000000,5.000000
OverallCond,3.991517,6.000033,3.991517,3.991517,3.991517,3.991517,3.991517,4.679500,3.991517,4.679500
YearBuilt,2003.000000,1976.000000,2001.000000,1915.000000,2000.000000,1993.000000,2004.000000,1973.000000,1931.000000,1939.000000
YearRemodAdd,2003.000000,1976.000000,2002.000000,1970.000000,2000.000000,1995.000000,2005.000000,1973.000000,1950.000000,1950.000000
MasVnrArea,19.433174,0.000000,17.768840,0.000000,25.404163,0.000000,18.962283,21.355330,0.000000,0.000000
BsmtFinSF1,144.117862,181.719186,110.441033,61.795315,136.624601,147.876483,230.755721,165.705771,0.000000,164.606644
BsmtFinSF2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.749196,0.000000,0.000000
BsmtUnfSF,29.991052,44.135410,56.896528,64.808848,61.166371,17.718036,47.147306,37.422800,90.703475,28.754166


### 10x5 CV

In [5]:
def tenx5_cv(model, X):
    i = 0
    five_times =np.zeros((5,))
    n_folds = 10
    rkf = RepeatedKFold(n_folds, n_repeats = 5).get_n_splits(X.values)
    rm= np.sqrt(-cross_val_score(model, X.values, y_train, scoring="neg_mean_squared_error", cv = rkf, verbose = 0, n_jobs = 3))
    print(rm.mean())

In [6]:
n_folds = 5

def rmsle_cv(model, X):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X.values)
    rmse= np.sqrt(-cross_val_score(model, X.values, y_train, scoring="neg_mean_squared_error", cv = kf, verbose = 2))
    return(rmse)

In [7]:
#X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train)
#print("X_tr:",X_tr.shape, "  X_val:", X_val.shape, " X_train:", X_train.shape," y_train:", y_train.shape )

#Root Mean Squared Error
from math import sqrt
from sklearn.metrics import mean_squared_error


def rmse(model, X, y): #input model, x is number of comparisons to display
    X_tr, X_val, y_tr, y_val = train_test_split(X.copy(), y.copy())
    #print("X_tr:",X_tr.shape, "  X_val:", X_val.shape, " X_train:", X_train.shape," y_train:", y_train.shape )
    model.fit(X_tr, y_tr)
    y_predicted = model.predict(X_val)    
    rms = sqrt(mean_squared_error(y_val, y_predicted))
    print("RMSE: %.4f " % (rms))
    print("")
    #for i in range (1,x):
     #   print( i,") Actual:    " ,np.expm1(y_val[i]))
      #  print( i,") Predicted: " ,np.expm1(y_predicted[i]))
       # print("")

In [8]:
#fits model to folds and returns column with predictions
def fit_folds(model, n_folds, X, y ):
    
    new_col = np.zeros((X.shape[0], 1))
    kfold = KFold( n_splits = n_folds, shuffle=False, random_state=4321) 
        
    for train_index, test_index in kfold.split(X):  
        Xt, Xv = X.iloc[train_index], X.iloc[test_index]
        yt, yv = y.iloc[train_index], y.iloc[test_index]
        model.fit(Xt, yt)
        pred = np.column_stack([model.predict(Xv)])        
        new_col[test_index] = pred
                        
    return(new_col)

## Models

In [9]:
lasso = make_pipeline(RobustScaler(), Lasso( alpha = .0002, fit_intercept = True, random_state = 4321)) # fit_intercept = True,

lasso_meta = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=4321))

GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, loss='huber', random_state =5)

model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,learning_rate=0.05, n_estimators=720, max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319, feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11, n_jobs = 1)

model_xgb =  xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, learning_rate=0.05, max_depth=3, min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571, subsample=0.5213, silent=1,random_state =7, nthread = -1, n_jobs = 1)

model_xgb_2 = xgb.XGBRegressor(learning_rate =0.01, n_estimators=3460, max_depth=3,min_child_weight=0 ,
                                     gamma=0, subsample=0.7,colsample_bytree=0.7,objective= 'reg:linear',nthread=4,
                                     scale_pos_weight=1,seed=27, reg_alpha=0.00006)

model_xgb_3 = make_pipeline(RobustScaler(), xgb.XGBRegressor( colsample_bytree=0.2, gamma=0.0, learning_rate=0.01, max_depth=4,
                     min_child_weight=1.5, n_estimators=7200, reg_alpha=0.9, reg_lambda=0.6, subsample=0.2, seed=4321,
                     silent=1))

RF_model = RandomForestRegressor(bootstrap = True,  max_depth = 15,  max_features = 0.5,  min_samples_leaf = 1,  n_estimators = 500,  oob_score = True,  random_state = 4321)


e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]
elastic_cv = make_pipeline(RobustScaler(), ElasticNetCV(max_iter=1e7, alphas=e_alphas, l1_ratio=e_l1ratio))

alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]

ridge_model2 = make_pipeline(RobustScaler(), RidgeCV(alphas = alphas_alt))

svr_opt = svm.SVR(C = 100000, gamma = 1e-08)

CBR_model = CatBoostRegressor(  logging_level= 'Silent', iterations=16226, bagging_temperature = 1,random_strength = 1, l2_leaf_reg=1, learning_rate =.017, rsm = 0.1, depth = 2, eval_metric = 'RMSE', random_seed=4321)
                              

## Hyperparameter Tuning

In [10]:
CBR_model_test = CatBoostRegressor(  logging_level= 'Silent', iterations=15000, bagging_temperature = 1,
                              random_strength = 1, l2_leaf_reg=1, learning_rate =.019, rsm = 0.1, 
                              depth = 2, eval_metric = 'RMSE', random_seed=4321)

In [20]:
rmse(lgb_2, meta_cols_f_2, y_train)

RMSE: 0.0765 



In [ ]:
tenx5_cv(CBR_model_test, meta_cols_f_2)

In [19]:
lgb_2 = lgb.LGBMRegressor(     max_bin   = 400, #50, 200
                               n_estimators=3000, #3000, 6000, 12000
                               learning_rate   = 0.01, # shrinkage_rate, .001
                               boosting_type   =  'dart', #gbdt
                               xgboost_dart_mode = True,
                               objective   =  'rmse' , #rmse, mae, huber, 
                               #metric   =  'l1' ,          # or  mae 
                               #sub_feature   = 0.345 ,   # feature_fraction (small values => use very different submodels)
                               #bagging_fraction   = 0.80 ,# sub_row
                               #bagging_freq   = 5,
                               #bagging_seed = 4321,
                               #max_depth = 5,   #8, 5
                               #feature_fraction = 0.9,
                               num_leaves   = 100  ,      # num_leaf, 100,50,30,20
                              # min_data_in_leaf = 22,  #22
                               #min_data   = 50 ,        # min_data_in_leaf
                               #min_hessian   = 0.05,     # min_sum_hessian_in_leaf
                               verbose   = 0,
                               #feature_fraction_seed   = 2,
                               #early_stopping_round = 100,
                               num_threads = 2,
                               bagging_seed   = 3)

In [21]:
lgb_grid_model = lgb.LGBMRegressor(boosting_type   =  'dart', #gbdt
                                   xgboost_dart_mode = True,
                                   objective   =  'rmse' )

In [56]:
lgb_grid = {"learning_rate": [.01],
                  "n_estimators": [ 2500, 3000, 3500, 4000],
                  "num_leaves": [51],
                  "max_bin":[ 300],
                  #"rsm": [.05, 0.1, 0.15],
                  "max_depth": [-1],
                  #"bootstrap": [True],
                  #"oob_score": [True],
                  #"n_estimators" :[500,1000,2000],
                  # "max_leaf_nodes" : np.rint(np.linspace(10, train_data.shape[0]/50, 10)).astype(int),
                  #"criterion": ["gini","entropy"],
                  'random_state' : [4321]}

In [57]:
from sklearn.model_selection import GridSearchCV
scoring = "neg_mean_squared_error"

gs = GridSearchCV(lgb_grid_model,
                     param_grid=lgb_grid,
                     scoring=scoring, cv=5, n_jobs= 3, verbose = 1)
gs.fit(meta_cols_f_2, y_train)
results = gs.cv_results_

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=3)]: Done  20 out of  20 | elapsed: 26.8min finished


In [58]:
# Best score
gs.best_score_

print('Best kfold score: {}'.format(gs.best_score_))
#print("\nMean score: {:.4f}".format(gs.score.mean()))
print('Best kfold parameters: {}'.format(gs.best_params_))

Best kfold score: -0.004301708209701528
Best kfold parameters: {'learning_rate': 0.01, 'max_bin': 300, 'max_depth': -1, 'n_estimators': 3500, 'num_leaves': 51, 'random_state': 4321}


### MLP_regressor

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split, learning_curve, GridSearchCV

# Neural Network
parameters = {'hidden_layer_sizes': [(100,100),(60,60),(50,50)], #(64,64) / "ranh"
              'activation': ['identity', 'logistic', 'tanh', 'relu']}
# create and fit a  model, testing random alpha values
mlp = GridSearchCV(MLPRegressor(), parameters)
mlp.fit(meta_cols_2, y_train)
# summarize the results of the random parameter search
print(mlp.best_score_)
#print(mean_squared_error(y_test,mlp.predict(meta_test_2)))
print(mlp.best_estimator_)

In [ ]:
mlp = MLPRegressor(activation='logistic', alpha=0.0001)

mlp = MLPRegressor(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

### tenx5_cv

In [67]:
lasso_test = make_pipeline(RobustScaler(), Lasso(alpha = .0003, fit_intercept = True, random_state = 4321))

In [68]:
tenx5_cv(lasso_test, meta_cols_f_2)

0.054300224193567524


In [ ]:
tenx5_cv(lasso_meta, meta_cols_2)

In [ ]:
model_lgb_test = lgb.LGBMRegressor(objective='regression',num_leaves=5,learning_rate=0.9, n_estimators=720, max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319, feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11, n_jobs = 1)

In [ ]:
tenx5_cv(model_lgb_test, meta_cols_2)

In [ ]:
GBoost_test = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.5, max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, loss='huber', random_state =5)

In [ ]:
tenx5_cv(GBoost_test, meta_cols_2)

In [ ]:
model_xgb_test =  xgb.XGBRegressor(learning_rate=0.9, max_depth=5, n_estimators=500, 
                               silent=1,random_state =4321,  n_jobs = 3)


In [ ]:
tenx5_cv(model_xgb_test, meta_cols_2)

### All CV

In [ ]:
score = rmsle_cv(mlp, meta_cols_2)
print("\nStack score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(lasso_meta, meta_cols_2)
print("\nStack score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(lasso, meta_cols_2)
print("\nStack score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(GBoost, meta_cols_2)
print("\nStack score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(model_lgb, meta_cols_2)
print("\nStack score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(model_xgb, meta_cols_2)
print("\nStack score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(model_xgb_2, meta_cols_2)
print("\nStack score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(model_xgb_3, meta_cols_2)
print("\nStack score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(CBR_model, meta_cols_2)
print("\nStack score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(elastic_cv, meta_cols_2)
print("\nStack score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(ridge_model2, meta_cols_2)
print("\nStack score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
score = rmsle_cv(svr_opt, meta_cols_2)
print("\nStack score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

### All RMSE

In [ ]:
rmse(CBR_model, meta_cols_2, y_train)

In [ ]:
rmse(lasso,meta_cols_2, y_train)

In [ ]:
rmse(lasso_meta,meta_cols_2, y_train)

In [ ]:
rmse(GBoost,meta_cols_2, y_train)

In [ ]:
rmse(model_lgb,meta_cols_2, y_train)

In [ ]:
rmse(model_xgb,meta_cols_2, y_train)

In [ ]:
rmse(model_xgb_2,meta_cols_2, y_train)

In [ ]:
rmse(model_xgb_3,meta_cols_2, y_train)

In [ ]:
rmse(RF_model,meta_cols_2, y_train)

In [ ]:
rmse(elastic_cv,meta_cols_2, y_train)

In [ ]:
rmse(ridge_model2,meta_cols_2, y_train)

In [ ]:
rmse(svr_opt,meta_cols_2, y_train)

## MLXTEND

In [ ]:
# mlextnd

from mlxtend.regressor import StackingCVRegressor

lasso = make_pipeline(RobustScaler(), Lasso( alpha = .0002, fit_intercept = True, random_state = 4321)) # fit_intercept = True,

lasso_meta = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=4321))

GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05, max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, loss='huber', random_state =5)

model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,learning_rate=0.05, n_estimators=720, max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319, feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11, n_jobs = 1)

model_xgb =  xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, learning_rate=0.05, max_depth=3, min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571, subsample=0.5213, silent=1,random_state =7, nthread = -1, n_jobs = 1)

model_xgb_2 = xgb.XGBRegressor(learning_rate =0.01, n_estimators=3460, max_depth=3,min_child_weight=0 ,
                                     gamma=0, subsample=0.7,colsample_bytree=0.7,objective= 'reg:linear',nthread=4,
                                     scale_pos_weight=1,seed=27, reg_alpha=0.00006)

model_xgb_3 = make_pipeline(RobustScaler(), xgb.XGBRegressor( colsample_bytree=0.2, gamma=0.0, learning_rate=0.01, max_depth=4,
                     min_child_weight=1.5, n_estimators=7200, reg_alpha=0.9, reg_lambda=0.6, subsample=0.2, seed=4321,
                     silent=1))

RF_model = RandomForestRegressor(bootstrap = True,  max_depth = 15,  max_features = 0.5,  min_samples_leaf = 1,  n_estimators = 500,  oob_score = True,  random_state = 4321)


e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]
elastic_cv = make_pipeline(RobustScaler(), ElasticNetCV(max_iter=1e7, alphas=e_alphas, l1_ratio=e_l1ratio))

alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]

ridge_model2 = make_pipeline(RobustScaler(), RidgeCV(alphas = alphas_alt))

svr_opt = svm.SVR(C = 100000, gamma = 1e-08)

CBR_model = CatBoostRegressor(  logging_level= 'Silent', iterations=16226, bagging_temperature = 1,random_strength = 1, l2_leaf_reg=1, learning_rate =.017, rsm = 0.1, depth = 2, eval_metric = 'RMSE', random_seed=4321)
#stack
stack_gen = StackingCVRegressor(regressors = ( lasso,  model_xgb_2, model_lgb,  ridge_model2 ), meta_regressor = lasso_meta,
                                            use_features_in_secondary=True) #use_features_in_secondary=True uses original dat and results
#model_xgb_2, model_lgb,
#prepare dataframes
stack_X = np.array(meta_cols_f) #default X_train
stack_y = np.array(y_train)

In [ ]:
mlxt_model = stack_gen.fit(stack_X, stack_y)

In [ ]:
#modified for stack_gen
n_folds = 5

def rmsle_cv_stack(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    rmse= np.sqrt(-cross_val_score(model, stack_X, stack_y, scoring="neg_mean_squared_error", cv = kf, verbose = 2))
    return(rmse)

In [ ]:
#stack_gen CV
score = rmsle_cv_stack(mlxt_model)
print("\nStack score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

#### Normalized Weight *Convert to function later

In [ ]:
kaggle_score = pd.DataFrame()

kaggle_score['Score'] = [1-.11567, 1-.11563, 1-.11436, 1-.11472, 1-.11302, 1-.11547, 1-.11643]
#print(kaggle_score)

scaler = MinMaxScaler(feature_range=(0.1, .99))
scaler.fit(kaggle_score)
#scaler.transform(kaggle_score)

kaggle_min_max = scaler.fit_transform(kaggle_score)
#print(kaggle_min_max)
kaggle_score['min_max']= kaggle_min_max
kaggle_score['Model'] = ['lasso_meta','Lasso','xgb','ridge', 'cbr', 'elastic', 'GBoost']
kaggle_score['weight']= (kaggle_score['min_max']/(kaggle_score['min_max'].sum(axis = 0)))*100
print(kaggle_score)

# Predictions

In [ ]:
#weighted_pred = (light_gbm_pred *0.40 +Lasso_pred * 0.25 + GBoost_pred * 0.25 + RF_pred * 0.10) *****Best***

In [ ]:
ensemble_pred = np.expm1(ensemble_model.predict(test.values))

In [ ]:
mlxt_pred = np.expm1(mlxt_model.predict(meta_test.values))   #****MLXT*****

### Averaged Meta

#test***************************************
lasso_test.fit(meta_cols_2, y_train)
lasso_test_pred = np.expm1(lasso_test.predict(meta_test_2.values))

In [27]:
lasso_meta.fit(meta_cols_f_2, y_train)
lasso_meta_pred = np.expm1(lasso_meta.predict(test_f_2.values))

In [28]:
lasso.fit(meta_cols_f_2, y_train)
lasso_pred = np.expm1(lasso.predict(test_f_2.values))

In [29]:
model_lgb.fit(meta_cols_f_2, y_train)
lgb_pred = np.expm1(model_lgb.predict(test_f_2.values))

In [30]:
model_xgb.fit(meta_cols_f_2, y_train)
xgb_pred = np.expm1(model_xgb.predict(test_f_2))

In [31]:
ridge_model2.fit(meta_cols_f_2, y_train)
ridge_pred = np.expm1(ridge_model2.predict(test_f_2.values))

In [32]:
CBR_model_test.fit(meta_cols_f_2, y_train)
CBR_model_test_pred = np.expm1(CBR_model_test.predict(test_f_2.values))

In [33]:
CBR_model.fit(meta_cols_f_2, y_train)
CBR_model_pred = np.expm1(CBR_model.predict(test_f_2.values))

In [34]:
elastic_cv.fit(meta_cols_f_2, y_train)
elastic_pred = np.expm1(elastic_cv.predict(test_f_2.values))

In [35]:
GBoost.fit(meta_cols_f_2, y_train)
GBoost_pred = np.expm1(GBoost.predict(test_f_2))

In [36]:
svr_opt.fit(meta_cols_f_2, y_train)
svr_opt_pred = np.expm1(svr_opt.predict(test_f_2))

In [37]:
stack_avg_pred = (lasso_meta_pred * 0.025 +   lasso_pred *0.025 +     xgb_pred *0.15  + ridge_pred *0.10
                 + CBR_model_test_pred *0.60+ elastic_pred *0.10 )

# Final Submission

In [38]:
sub_1 = pd.DataFrame()
sub_1['Id'] = test_ID
sub_1['SalePrice'] = stack_avg_pred
sub_1.to_csv('submission_98.csv',index=False)